In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df_train = pd.read_csv(r"drive/My Drive/bitrate_model_final/bitrate_training.csv")
df_test = pd.read_csv(r"drive/My Drive/bitrate_model_final/bitrate_testing.csv")
df_validation = pd.read_csv(r"drive/My Drive/bitrate_model_final/bitrate_validation.csv")

In [3]:
#pd.set_option('display.max_rows', 1000)
#df_train.isna().sum()

In [4]:
#df_validation
df_train = df_train.iloc[: , 1:]
df_test = df_test.iloc[: , 1:]
df_validation = df_validation.iloc[: , 1:]

In [5]:
df_train["text"] = df_train["text"].str.lower()
df_test["text"] = df_test["text"].str.lower()
df_validation["text"] = df_validation["text"].str.lower()

In [6]:
df_train["len_text"] = df_train['text'].str.len() #df_test
df_test["len_text"] = df_test['text'].str.len()
df_validation["len_text"] = df_validation['text'].str.len()


In [7]:
df_train.columns

Index(['filename', 'size-bytes', 'height', 'width', 'framespersecond',
       'duration', 'i_frame', 'b_frame', 'p_frame', 'text', 'Bitrate',
       'bit_rate_mode', 'channel_layout', 'channel_positions', 'channel_s',
       'frame_count', 'proportion_of_this_stream', 'stream_size',
       'other_bit_rate_mode', 'len_text'],
      dtype='object')

In [8]:
df_train["other_bit_rate_mode"].value_counts()

Constant    595
Variable    393
Name: other_bit_rate_mode, dtype: int64

In [9]:
df_train["bit_rate_mode"].value_counts()

CBR    595
VBR    393
Name: bit_rate_mode, dtype: int64

In [10]:
df_train.drop(columns=['filename', 'text','bit_rate_mode'], axis=1, inplace=True)
df_test.drop(columns=['filename', 'text','bit_rate_mode'], axis=1, inplace=True)
df_validation.drop(columns=['filename', 'text','bit_rate_mode'], axis=1, inplace=True)


In [11]:
#df_train.isna().sum()
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_validation = df_validation.reset_index()

In [12]:
#df_train["label"].replace({"CBR": 1, "VBR": 0}, inplace=True)
encode_train = pd.get_dummies(df_train, columns = ['channel_layout','channel_positions','other_bit_rate_mode'])
encode_test = pd.get_dummies(df_test, columns = ['channel_layout','channel_positions','other_bit_rate_mode'])
encode_val = pd.get_dummies(df_validation, columns = ['channel_layout','channel_positions','other_bit_rate_mode'])


In [13]:
#channel_layout_C L R Ls Rs LFE  channel_positions_Front:L C R, Side: L R, LFE

encode_train.drop(columns=['channel_layout_C L R Ls Rs LFE', 'channel_positions_Front: L C R, Side: L R, LFE'], axis=1, inplace=True)
#encode_test.drop(columns=['channel_layout_C L R Ls Rs LFE', 'channel_positions_Front: L C R, Side: L R, LFE'], axis=1, inplace=True)
encode_val.drop(columns=['channel_layout_C L R Ls Rs LFE', 'channel_positions_Front: L C R, Side: L R, LFE'], axis=1, inplace=True)


In [14]:
df = pd.concat([encode_train, encode_test,encode_val])

In [15]:
df

,index,size-bytes,height,width,framespersecond,duration,i_frame,b_frame,p_frame,Bitrate,channel_s,frame_count,proportion_of_this_stream,stream_size,len_text,channel_layout_C,channel_layout_L R,channel_positions_Front: C,channel_positions_Front: L R,other_bit_rate_mode_Constant,other_bit_rate_mode_Variable
0,0,125469259.0,144.0,256.0,25,5202.640000,2255,64571,63240,192,2,224065,0.66347,83244466,6395,0,1,0,1,1,0
1,1,156812882.0,240.0,426.0,23,6375.608696,2536,70413,73690,205,2,263400,0.62404,97858176,12299,0,1,0,1,1,0
2,2,518875095.0,546.0,1280.0,24,4836.000000,2185,63281,50598,858,2,208270,0.14912,77376319,11292,0,1,0,1,0,1
3,3,520279708.0,546.0,1280.0,24,4836.000000,2185,63281,50598,860,2,208273,0.14872,77377433,11289,0,1,0,1,1,0
4,4,282485177.0,342.0,640.0,24,5006.916667,1864,79409,38893,451,2,215632,0.21270,60083583,11336,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,193,80518345.0,720.0,1280.0,23,618.130435,236,8242,5739,1086,2,25537,0.11783,9487487,980,0,1,0,1,0,1
194,194,243886613.0,1080.0,1920.0,23,636.956522,300,7364,6986,3192,2,26317,0.04009,9777272,697,0,1,0,1,1,0
195,195,62480217.0,720.0,1280.0,29,302.896552,110,3324,5350,1705,2,12623,0.07365,4601535,114,0,1,0,1,0,1
196,196,21342015.0,668.0,1280.0,23,134.391304,34,1915,1142,1324,2,5553,0.09485,2024311,359,0,1,0,1,0,1


In [16]:
df = df.iloc[: , 1:]

In [17]:
df

,size-bytes,height,width,framespersecond,duration,i_frame,b_frame,p_frame,Bitrate,channel_s,frame_count,proportion_of_this_stream,stream_size,len_text,channel_layout_C,channel_layout_L R,channel_positions_Front: C,channel_positions_Front: L R,other_bit_rate_mode_Constant,other_bit_rate_mode_Variable
0,125469259.0,144.0,256.0,25,5202.640000,2255,64571,63240,192,2,224065,0.66347,83244466,6395,0,1,0,1,1,0
1,156812882.0,240.0,426.0,23,6375.608696,2536,70413,73690,205,2,263400,0.62404,97858176,12299,0,1,0,1,1,0
2,518875095.0,546.0,1280.0,24,4836.000000,2185,63281,50598,858,2,208270,0.14912,77376319,11292,0,1,0,1,0,1
3,520279708.0,546.0,1280.0,24,4836.000000,2185,63281,50598,860,2,208273,0.14872,77377433,11289,0,1,0,1,1,0
4,282485177.0,342.0,640.0,24,5006.916667,1864,79409,38893,451,2,215632,0.21270,60083583,11336,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,80518345.0,720.0,1280.0,23,618.130435,236,8242,5739,1086,2,25537,0.11783,9487487,980,0,1,0,1,0,1
194,243886613.0,1080.0,1920.0,23,636.956522,300,7364,6986,3192,2,26317,0.04009,9777272,697,0,1,0,1,1,0
195,62480217.0,720.0,1280.0,29,302.896552,110,3324,5350,1705,2,12623,0.07365,4601535,114,0,1,0,1,0,1
196,21342015.0,668.0,1280.0,23,134.391304,34,1915,1142,1324,2,5553,0.09485,2024311,359,0,1,0,1,0,1


In [18]:
df.reset_index(inplace=True,drop=True)

In [19]:
df.isna().sum()

size-bytes                      0
height                          0
width                           0
framespersecond                 0
duration                        0
i_frame                         0
b_frame                         0
p_frame                         0
Bitrate                         0
channel_s                       0
frame_count                     0
proportion_of_this_stream       0
stream_size                     0
len_text                        0
channel_layout_C                0
channel_layout_L R              0
channel_positions_Front: C      0
channel_positions_Front: L R    0
other_bit_rate_mode_Constant    0
other_bit_rate_mode_Variable    0
dtype: int64

In [20]:
y_train = df["Bitrate"]
y_train=y_train.astype('int')
df.drop("Bitrate", axis=1, inplace=True)

X_train = df


In [21]:
#from sklearn.model_selection import train_test_split
#x_train,x_test,y_train,y_test = train_test_split(X_train,y_train,test_size = 0.25)


In [22]:
!pip install xgboost

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, cross_val_predict
import numpy as np
#xgbr = XGBRegressor(verbosity=0) 
xgbr = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)
cv_r2_scores_xg = cross_val_score(xgbr, X_train, y_train, cv=10,scoring='neg_root_mean_squared_error')

print(cv_r2_scores_xg)
print("Mean 10-Fold RMSE: {}".format(abs(np.mean(cv_r2_scores_xg))))


[09:07:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:07:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:07:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:07:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:07:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:07:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:07:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:07:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [ ]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 0)
#from sklearn.metrics import neg_root_mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, cross_val_predict
import numpy as np
cv_r2_scores_rf = cross_val_score(regressor, X_train, y_train, cv=10,scoring='neg_root_mean_squared_error')

print(cv_r2_scores_rf)
print("Mean 10-Fold RMSE: {}".format(abs(np.mean(cv_r2_scores_rf))))


[-56.76513521 -27.63469303 -40.36853608 -20.96045276 -95.80452623
 -33.48902048 -44.07585688 -38.39733158 -40.68507542 -51.2616194 ]
Mean 10-Fold RMSE: 44.9442247066505


In [ ]:
from sklearn.ensemble import RandomForestRegressor
clfr=RandomForestRegressor(n_estimators=100)

cv_r2_scores_rf1 = cross_val_score(clfr, X_train, y_train, cv=10,scoring='neg_root_mean_squared_error')

print(cv_r2_scores_rf1)
print("Mean 10-Fold RMSE : {}".format(abs(np.mean(cv_r2_scores_rf1))))

[ -17.99981445  -41.51818151  -15.0001895   -21.64301743 -132.54789714
  -34.21184582  -42.18226586  -13.89412453  -39.96236127  -76.73251497]
Mean 10-Fold RMSE : 43.569221246150136
